In [95]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import copy
import math
from scipy import stats
from scipy.signal import butter, lfilter
import statistics
import scipy.linalg as la
from scipy.signal import periodogram
os.chdir(r'F:\KARTIK\2021\Freezing of Gait')

In [96]:
data_path = os.path.join(
                    os.getcwd(),
                    'dataset_fog_release',
                    'dataset'
                    )
if not os.path.exists(data_path):
    print("Path Error")

In [9]:
def label_prefog(dataset, window_length = 1):
    # Drop all the rows for which Action is 0 or the rows which are not part of the experiment
    dataset.drop(index = list(dataset[dataset['Action'] == 0].index),
                 inplace = True)
    
    window_length = 64*window_length
    
    fog_index = []
    for i in dataset.index:
        if dataset.loc[i, 'Action'] == 2:
            fog_index.append(i)
    fog_index
    
    
    start_indices = []
    for i in fog_index:
        if (dataset.loc[i-1, 'Action'] != dataset.loc[i, 'Action']):
            start_indices.append(i)
    
    prefog = []
    for start in start_indices:
        prefog_start = [x for x in range(start-window_length, start)]
        prefog.append(prefog_start)
        
    prefog = [item for sublist in prefog for item in sublist]
    
    for i in prefog:
        dataset.loc[i,'Action'] = 3
    dataset['Action'] = dataset['Action'] - 1
    return dataset

In [24]:
people = []

for person in os.listdir(data_path):
    if '.txt' in person:
        people.append(person)

for window_length in range(1,2):
    dataset = pd.DataFrame()
    for person in people:
        name = person.split('R')[0]
        print(name)
        file = data_path + "\\" + person
        temp = pd.read_csv(file, delimiter = " ", header = None)
        print(person, ' is read', end = "\t")
        
        if 2 in temp[max(temp.columns)].unique():
            print("Adding {} to dataset".format(person), end = "\t")
            temp.columns = [
                            'time',
                            'A_F', 'A_V', 'A_L',
                            'L_F', 'L_V', 'L_L',
                            'T_F', 'T_V', 'T_L',
                            'Action'                           
                           ]
            temp = label_prefog(temp, window_length).reset_index(drop = True)
            #temp['name'] = name
            print("{} is labelled".format(person))
            
            to_name = os.getcwd() + "\\APPROACH - 2" + "\\raw_labelled" + "\\" + person[:6]  +  ".csv"
            
            temp.to_csv(to_name, index = False)
                
#             dataset = pd.concat([dataset, temp],axis = 0)
            
#         print('')
        
#         dataset.reset_index(drop = True, inplace = True)
#         to_path = data_path + "\\raw_labelled"
#         to_name = to_path + "\\win_" + str(window_length) + ".csv"
        
#         dataset.to_csv(to_name, index = False)

# display(dataset.head())   
        

S01
S01R01.txt  is read	Adding S01R01.txt to dataset	S01R01.txt is labelled
S01
S01R02.txt  is read	Adding S01R02.txt to dataset	S01R02.txt is labelled
S02
S02R01.txt  is read	Adding S02R01.txt to dataset	S02R01.txt is labelled
S02
S02R02.txt  is read	Adding S02R02.txt to dataset	S02R02.txt is labelled
S03
S03R01.txt  is read	Adding S03R01.txt to dataset	S03R01.txt is labelled
S03
S03R02.txt  is read	Adding S03R02.txt to dataset	S03R02.txt is labelled
S03
S03R03.txt  is read	S04
S04R01.txt  is read	S05
S05R01.txt  is read	Adding S05R01.txt to dataset	S05R01.txt is labelled
S05
S05R02.txt  is read	Adding S05R02.txt to dataset	S05R02.txt is labelled
S06
S06R01.txt  is read	Adding S06R01.txt to dataset	S06R01.txt is labelled
S06
S06R02.txt  is read	S07
S07R01.txt  is read	Adding S07R01.txt to dataset	S07R01.txt is labelled
S07
S07R02.txt  is read	Adding S07R02.txt to dataset	S07R02.txt is labelled
S08
S08R01.txt  is read	Adding S08R01.txt to dataset	S08R01.txt is labelled
S09
S09R01.txt  

Extracting non overlapping windows of length w * f_s from the accelerometer dataset

In [25]:
def create_window(act,window_length,dataframe):
    
   # Get the indices of all the 0s in first loop, then all the 1s in the second and lastly, all the 2s in the third loop 
  indices = list(dataframe[dataframe.Action == act].index)
  groups = []
  temp = []
  group_count = 0
  for i in range(len(indices)):
        
    if i == len(indices)-1:                  # If it is the last value
       
      temp.append(indices[i])                # Add the last value
      groups.append(temp)
      temp = []
      break
    temp.append(indices[i])
    
    if indices[i]+1 != indices[i+1]:       # If consecutive values are not same
      group_count+=1
      groups.append(temp)
      temp = []

  fs = 64
  window_length = 1
  # window_length = window_length*fs

  final_dataframe = pd.DataFrame()
  for i in groups: 
    required = math.floor(len(i)/(window_length*fs))
    
    req_index = i[0:(required*fs)]
    
    final_dataframe = pd.concat([final_dataframe,dataframe.iloc[req_index,:]],axis = 0)
  return final_dataframe
            

In [51]:
for window_length in range(1,2):
    path = os.getcwd() + "\\APPROACH - 2" + "\\raw_labelled"
      #name = path+"/raw_labelled/win_"+str(window_length)+".csv"
    for name in os.listdir(path):
        dataframe = pd.read_csv(os.path.join(path, name))

        activities = []
        for act in range(3):
            activities.append(create_window(act,window_length,dataframe))
            to_write = pd.concat(activities,axis = 0)
            to_path = os.getcwd() + "\\APPROACH - 2"  + "/window"+"/windowed_"+ name[:-4] +".csv"
            to_write.to_csv(to_path,index = False)

## Features to be extracted:
1. Minimum
2. Maximum
3. Median
4. Mean
5. ArmMean
6. Root Mean Square (RMS)
7. GeoMean
8. Variance
9. Standard Deviation
10. Kurtosis
11. Skewness
12. Mode
13. TrimMean
14. Entropy
15. Asymmetry Coefficient
16. Zero Crossing Rate (ZCR)
17. Mean Crossing Rate (MCR)
18. Range


In [46]:
def zcr(array):
    array = np.array(array)
    x =  np.diff((array) > 0)
    return len(np.nonzero(x)[0])

def getZeroCrossingRate(arr):
    my_array = np.array(arr)
    return float("{0:.2f}".format((((my_array[:-1] * my_array[1:]) < 0).sum())/len(arr)))

def getMeanCrossingRate(arr):
    return getZeroCrossingRate(np.array(arr) - np.mean(arr))

def getMode(arr):
    l = []
    for ele in arr:
        l.append(ele)
    return statistics.mode(l)

def geometric_mean(values):
    for i in range(0, len(values)):
        if values[i] == 0:
            del values[i]
        if len(values) == 0:
            return 0
    return np.prod([x for x in values]) ** (1 / len([x for x in values]))

In [59]:
window_length = 1
f_s = 64
w = window_length * f_s

FE_path = os.getcwd() + "\\APPROACH - 2"  + "\\window"

for file in os.listdir(FE_path):
    #print(file)
    print("Opened File: " , file, end  = "\n")
    name = os.path.join(FE_path, file)
    dataframe = pd.read_csv(name)
    df = dataframe.drop(columns = ['time', 'Action'])
    statistics = pd.DataFrame()

    col = list(df.columns)
    for column in col:
        print(file, "\t", column)

        f1 = []
        f2 = []
        f3 = []
        f4 = []
        f5 = []
        f6 = []
        f7 = []
        f8 = []
        f9 = []
        f10 = []
        f11 = []
        f12 = []
        f13 = []
        f14 = []
        f15 = []
        f16 = []
        f17 = []
        f18 = []

        for i in range(0, len(df), w):
            f_1 = min(df[column].iloc[i:i+w])
            f_2 = max(df[column].iloc[i:i+w])
            f_3 = np.median(df[column].iloc[i:i+w])
            f_4 = np.mean(df[column].iloc[i:i+w])
            f_5 = stats.hmean(abs(df[column].iloc[i:i+w]))

            f_6 = np.sqrt(np.mean(df[column].iloc[i:i+w]**2))

    #        f_7 = geometric_mean(df[column].iloc[i:i+w])
            f_8 = np.var(df[column].iloc[i:i+w])
            f_9 = np.std(df[column].iloc[i:i+w])
            f_10 = stats.kurtosis(df[column].iloc[i:i+w])
            f_11 = stats.skew(df[column].iloc[i:i+w])
            #f_12 = getMode(df[column].iloc[i:i+w])
            f_13 = stats.tmean(df[column].iloc[i:i+w])
            #f_14 = stats.entropy(df[column].iloc[i:i+w])
            f_15 = df[column].iloc[i] / np.std(df[column].iloc[i:i+w])
            f_16 = getZeroCrossingRate(df[column].iloc[i:i+w])
            f_17 = getMeanCrossingRate(df[column].iloc[i:i+w])
            f_18 = f_2 - f_1


            f1.append(f_1)
            f2.append(f_2)
            f3.append(f_3)
            f4.append(f_4)
            f5.append(f_5)
            f6.append(f_6)
            #f7.append(f_7)
            f8.append(f_8)
            f9.append(f_9)
            f10.append(f_10)
            f11.append(f_11)
    #         f12.append(f_12)
            f13.append(f_13)
            #f14.append(f_14)
            f15.append(f_15)
            f16.append(f_16)
            f17.append(f_17)
            f18.append(f_18)

        statistics['Minimum_' + column] = f1
        statistics['Maximum_' + column] = f2
        statistics['Median_' + column] = f3
        statistics['Mean_' + column] = f4
        statistics['ArmMean_' + column] = f5
        statistics['RMS_' + column] = f6
        #statistics['GeometricMean_' + column] = f7
        statistics['Variance_' + column] = f8
        statistics['STD_' + column] = f9
        statistics['Kurtosis_' + column] = f10
        statistics['Skewness_' + column] = f11
        #statistics['Mode_' + column] = f12
        statistics['TrimMean_' + column] = f13
        #statistics['Entropy_' + column] = f14
        statistics['AsymmetryCoefficient_' + column] = f15
        statistics['ZCR_' + column] = f16
        statistics['MCR_' + column] = f17
        statistics['Range' + column] = f18    

    print("The shape of the dataset is: ", statistics.shape)
    statistics1 = copy.copy(statistics)
    statistics1['w'] = dataframe['Action'].iloc[[x for x in range(0, len(dataframe), w)]].to_list()
    order = ['w']
    order += statistics1.columns.to_list()[:-1]
    statistics1 = statistics1[order]
    statistics1.columns
    col = statistics1.columns.to_list()
    col[0] = 0
    statistics1.columns = col
    feature_name = os.getcwd() + "\\APPROACH - 2"  + "/features"+"/time_"+ file[:-4] +".csv"
    print("Saving File: ", file)
    statistics1.to_csv(feature_name, index = False) 

Opened File:  windowed_S01R01.csv
windowed_S01R01.csv 	 A_F
windowed_S01R01.csv 	 A_V
windowed_S01R01.csv 	 A_L
windowed_S01R01.csv 	 L_F
windowed_S01R01.csv 	 L_V
windowed_S01R01.csv 	 L_L
windowed_S01R01.csv 	 T_F
windowed_S01R01.csv 	 T_V
windowed_S01R01.csv 	 T_L
The shape of the dataset is:  (1434, 135)
Saving File:  windowed_S01R01.csv
Opened File:  windowed_S01R02.csv
windowed_S01R02.csv 	 A_F
windowed_S01R02.csv 	 A_V
windowed_S01R02.csv 	 A_L
windowed_S01R02.csv 	 L_F
windowed_S01R02.csv 	 L_V
windowed_S01R02.csv 	 L_L
windowed_S01R02.csv 	 T_F
windowed_S01R02.csv 	 T_V
windowed_S01R02.csv 	 T_L
The shape of the dataset is:  (446, 135)
Saving File:  windowed_S01R02.csv
Opened File:  windowed_S02R01.csv
windowed_S02R01.csv 	 A_F
windowed_S02R01.csv 	 A_V
windowed_S02R01.csv 	 A_L
windowed_S02R01.csv 	 L_F
windowed_S02R01.csv 	 L_V
windowed_S02R01.csv 	 L_L
windowed_S02R01.csv 	 T_F
windowed_S02R01.csv 	 T_V
windowed_S02R01.csv 	 T_L
The shape of the dataset is:  (392, 135)
Savi

c:\python37\lib\site-packages\ipykernel_launcher.py:55: RuntimeWarning: divide by zero encountered in true_divide


windowed_S06R01.csv 	 A_V
windowed_S06R01.csv 	 A_L
windowed_S06R01.csv 	 L_F
windowed_S06R01.csv 	 L_V
windowed_S06R01.csv 	 L_L
windowed_S06R01.csv 	 T_F
windowed_S06R01.csv 	 T_V
windowed_S06R01.csv 	 T_L
The shape of the dataset is:  (1670, 135)
Saving File:  windowed_S06R01.csv
Opened File:  windowed_S07R01.csv
windowed_S07R01.csv 	 A_F
windowed_S07R01.csv 	 A_V
windowed_S07R01.csv 	 A_L
windowed_S07R01.csv 	 L_F
windowed_S07R01.csv 	 L_V
windowed_S07R01.csv 	 L_L
windowed_S07R01.csv 	 T_F
windowed_S07R01.csv 	 T_V
windowed_S07R01.csv 	 T_L
The shape of the dataset is:  (1145, 135)
Saving File:  windowed_S07R01.csv
Opened File:  windowed_S07R02.csv
windowed_S07R02.csv 	 A_F
windowed_S07R02.csv 	 A_V
windowed_S07R02.csv 	 A_L
windowed_S07R02.csv 	 L_F
windowed_S07R02.csv 	 L_V
windowed_S07R02.csv 	 L_L
windowed_S07R02.csv 	 T_F
windowed_S07R02.csv 	 T_V
windowed_S07R02.csv 	 T_L
The shape of the dataset is:  (442, 135)
Saving File:  windowed_S07R02.csv
Opened File:  windowed_S08R01

In [61]:
window_length = 1
f_s = 64
w = window_length * f_s

FE_path = os.getcwd() + "\\APPROACH - 2"  + "\\window"

for file in os.listdir(FE_path):
    #print(file)
    print("Opened File: " , file, end  = "\n")
    name = os.path.join(FE_path, file)
    dataframe = pd.read_csv(name)
    df = dataframe.drop(columns = ['time', 'Action'])
    fi = pd.DataFrame()
    power = pd.DataFrame()
    order = 5
    bands = {'locomotor' : (0.5,3), 'freeze' : (3,8)}
    col = list(df.columns)


    for column in col:
        xtemp = []
        xtemp1 = []
        for i in range(0, len(df),w):
            nyq = 0.5*f_s             # Nyquist Rate

            # LOCOMOTOR HAS A FREQUENCY RANGE OF 0.5Hz - 3Hz FOR TOTAL POWER

            loc_low = 0.5 / nyq
            loc_high = 3 / nyq

            # Clipping off band from the window

            b, a = butter(order, [loc_low, loc_high], btype = 'band')
            y = lfilter(b, a, df[column].iloc[i:i+w])

            # Total power in locomotor band

            e1 = sum([x**2 for x in y])




            # FREEZING BAND HAS A FREQUENCY RANGE OF 3Hz - 8Hz

            frez_low = 3 / nyq
            frez_high = 8 / nyq

            # Clipping off band from the window

            b1, a1 = butter(order, [frez_low, frez_high], btype = 'band')
            y1 = lfilter(b1, a1, df[column].iloc[i:i+w])

            # Total power in freezing band

            e2 = sum([x**2 for x in y1])

            freezing_index = e2/e1
            pwr = e2 + e1

            xtemp.append(freezing_index)
            xtemp1.append(pwr)

        print(column + " done", end = " ")

        fi['FI_' + column] = xtemp
        power['P_' + column] = xtemp1

    print(" Freezing Index and Power has been calculated")

    E = []

    for i in range(0, len(df), w):
        energy = np.sum((df.iloc[i:i + w,:])**2)
        E.append(energy)

    E = pd.DataFrame(E)

    E.columns = ["EN_" + x for x in df.columns]

    peak_f = pd.DataFrame()
    PSE = pd.DataFrame()
    for s in col:
      peakF = []
      pse = []
      for i in range(0,len(df),w):
          f,Pxx_den = periodogram(df[s].iloc[i:i+w],w)
          p_norm = Pxx_den/sum(Pxx_den)
          p_norm = list(filter(lambda a: a != 0, p_norm))
          pse.append(-(np.sum(p_norm*np.log(p_norm))))
          peak = (f_s/w)*max(Pxx_den)
          peakF.append(peak)
      PSE['ENt_'+s] = pse
      peak_f['peak_'+s] = peakF
    PSE.fillna(0,inplace = True)

    freq = pd.concat([fi, power, E, peak_f, PSE], axis = 1)

    print(freq.shape)
    feature_name = os.getcwd() + "\\APPROACH - 2"  + "/features"+"/freq_"+ file[:-4] +".csv"

    freq.to_csv(feature_name, index = False)  

Opened File:  windowed_S01R01.csv
A_F done A_V done A_L done L_F done L_V done L_L done T_F done T_V done T_L done  Freezing Index and Power has been calculated
(1434, 45)
Opened File:  windowed_S01R02.csv
A_F done A_V done A_L done L_F done L_V done L_L done T_F done T_V done T_L done  Freezing Index and Power has been calculated
(446, 45)
Opened File:  windowed_S02R01.csv
A_F done A_V done A_L done L_F done L_V done L_L done T_F done T_V done T_L done  Freezing Index and Power has been calculated
(392, 45)
Opened File:  windowed_S02R02.csv
A_F done A_V done A_L done L_F done L_V done L_L done T_F done T_V done T_L done  Freezing Index and Power has been calculated
(1001, 45)
Opened File:  windowed_S03R01.csv
A_F done A_V done A_L done L_F done L_V done L_L done T_F done T_V done T_L done  Freezing Index and Power has been calculated
(1379, 45)
Opened File:  windowed_S03R02.csv
A_F done A_V done A_L done L_F done L_V done L_L done T_F done T_V done T_L done  Freezing Index and Power h

c:\python37\lib\site-packages\ipykernel_launcher.py:87: RuntimeWarning: invalid value encountered in true_divide


(1670, 45)
Opened File:  windowed_S07R01.csv
A_F done A_V done A_L done L_F done L_V done L_L done T_F done T_V done T_L done  Freezing Index and Power has been calculated
(1145, 45)
Opened File:  windowed_S07R02.csv
A_F done A_V done A_L done L_F done L_V done L_L done T_F done T_V done T_L done  Freezing Index and Power has been calculated
(442, 45)
Opened File:  windowed_S08R01.csv
A_F done A_V done A_L done L_F done L_V done L_L done T_F done T_V done T_L done  Freezing Index and Power has been calculated
(759, 45)
Opened File:  windowed_S09R01.csv
A_F done A_V done A_L done L_F done L_V done L_L done T_F done T_V done T_L done  Freezing Index and Power has been calculated
(1711, 45)


In [97]:
data_path_1 = os.getcwd() + "\\APPROACH - 2"  + "\\features" + "\\frequency"
fr = os.listdir(data_path_1)

data_path_2 = os.getcwd() + "\\APPROACH - 2"  + "\\features" + "\\time"
ti = os.listdir(data_path_2)

total_files = len(fr)
print(ti)

['time_windowed_S01R01.csv', 'time_windowed_S01R02.csv', 'time_windowed_S02R01.csv', 'time_windowed_S02R02.csv', 'time_windowed_S03R01.csv', 'time_windowed_S03R02.csv', 'time_windowed_S05R01.csv', 'time_windowed_S05R02.csv', 'time_windowed_S06R01.csv', 'time_windowed_S07R01.csv', 'time_windowed_S07R02.csv', 'time_windowed_S08R01.csv', 'time_windowed_S09R01.csv']


In [99]:
for i in range(13):
    fr_path = os.path.join(data_path_1, fr[i])
    ti_path = os.path.join(data_path_2, ti[i])
    if os.path.exists(fr_path) and os.path.exists(ti_path):
        fr_file = pd.read_csv(fr_path)
        ti_file = pd.read_csv(ti_path)
        
        final_features = pd.concat([ti_file, fr_file], axis = 1)
        #print(final_features.head(1))
        print(fr[i][-10:-4])
        
        feature_name = os.getcwd() + "\\APPROACH - 2"  +  "\\final_features" + fr[i][-10:-4] +  ".csv"
        
        final_features.to_csv(feature_name)

S01R01
S01R02
S02R01
S02R02
S03R01
S03R02
S05R01
S05R02
S06R01
S07R01
S07R02
S08R01
S09R01
